Make csv tables
GtC/yr for CO2

Mt/yr for CH4, N2O (in units of N2 – don’t ask me why), SOx (in units of S), CO, NMVOC, NOx (in units of N), BC, OC, NH3
kt/yr for everything else.

Concentrations are ppm for CO2, ppb for CH4 and N2O, ppt for everything else.

constants module

In [1]:
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as pl
import os
import urllib
import json
from matplotlib import rc
from matplotlib.ticker import AutoMinorLocator
from scipy.stats import gaussian_kde
from scipy.optimize import root
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
from fair.forcing.ghg import etminan
from fair.constants import molwt

In [2]:
import sys
sys.path.append('/Users/earpmf/PYTHON/twolayermodel/')
sys.path.append('/Users/earpmf/PYTHON/AR6_aerosols/scripts')
print (sys.path)

['/Users/earpmf/OneDrive - University of Leeds/PYTHON/Corinne', '/Users/earpmf/opt/anaconda3/lib/python37.zip', '/Users/earpmf/opt/anaconda3/lib/python3.7', '/Users/earpmf/opt/anaconda3/lib/python3.7/lib-dynload', '', '/Users/earpmf/opt/anaconda3/lib/python3.7/site-packages', '/Users/earpmf/opt/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/earpmf/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/earpmf/.ipython', '/Users/earpmf/PYTHON/twolayermodel/', '/Users/earpmf/PYTHON/AR6_aerosols/scripts']


In [3]:
emissions_g2_2=np.load('/Users/earpmf/PYTHON/Piers-climate/data/emissions_ndc_month_g2_2.npy')
emissions_g1_3=np.load('/Users/earpmf/PYTHON/Piers-climate/data/emissions_ndc_month_g1_3.npy')
emissions_g3=np.load('/Users/earpmf/PYTHON/Piers-climate/data/emissions_ndc_month_g3.npy')
emissions_base=np.load('/Users/earpmf/PYTHON/Piers-climate/data/emissions_ndc_month_base.npy')
emissions_g4=np.load('/Users/earpmf/PYTHON/Piers-climate/data/emissions_ndc_month_g4.npy')
Fsolar_month=np.load('/Users/earpmf/PYTHON/Piers-climate/data/solarf_month.npy')
Fvolcanic_month=np.load('/Users/earpmf/PYTHON/Piers-climate/data/volcanicf_month.npy')
naturale_month=np.load('/Users/earpmf/PYTHON/Piers-climate/data/naturale_month.npy')
                       
                       
pd.DataFrame(emissions_g2_2).to_csv("outputs/emissions_ndc_month_g2_2.csv")
pd.DataFrame(emissions_g1_3).to_csv("outputs/emissions_ndc_month_g1_3.csv")
pd.DataFrame(emissions_g3).to_csv("outputs/emissions_ndc_month_g3.csv")
pd.DataFrame(emissions_base).to_csv("outputs/emissions_ndc_month_base.csv")
pd.DataFrame(emissions_g3).to_csv("outputs/emissions_ndc_month_g4.csv")

In [4]:
from fair.ancil import natural, cmip6_volcanic, cmip6_solar, historical_scaling
n=natural.Emissions.emissions
print(n[300,:])
print (cmip6_volcanic.Forcing.volcanic.shape)

import sys
sys.path.append('/Users/earpmf/PYTHON/FAIR/')
#sys.path.append('/Users/earpmf/PYTHON/AR6_aerosols/scripts')
#print (sys.path)

import fair_mod
#import fair
fair_mod.__version__

[190.5807   8.9883]
(736,)


'1.5+0.g152cd0d.dirty'

In [5]:
C22, F22, T22 = fair_mod.forward.fair_scm(emissions=emissions_g2_2,
                                             F_volcanic=Fvolcanic_month,F_solar=Fsolar_month,
                                             natural=naturale_month)
Cbase, Fbase, Tbase = fair_mod.forward.fair_scm(emissions=emissions_base,
                                             F_volcanic=Fvolcanic_month,F_solar=Fsolar_month,
                                             natural=naturale_month)
C13, F13, T22 = fair_mod.forward.fair_scm(emissions=emissions_g1_3,
                                             F_volcanic=Fvolcanic_month,F_solar=Fsolar_month,
                                             natural=naturale_month)
C3, F3, T3 = fair_mod.forward.fair_scm(emissions=emissions_g3,
                                             F_volcanic=Fvolcanic_month,F_solar=Fsolar_month,
                                             natural=naturale_month)
C4, F4, T4 = fair_mod.forward.fair_scm(emissions=emissions_g4,
                                             F_volcanic=Fvolcanic_month,F_solar=Fsolar_month,
                                             natural=naturale_month)

<class 'numpy.ndarray'> [1.6  2.75]
fggfgfgfg_test2
<class 'numpy.ndarray'> [1.6  2.75]
fggfgfgfg_test2
<class 'numpy.ndarray'> [1.6  2.75]
fggfgfgfg_test2
<class 'numpy.ndarray'> [1.6  2.75]
fggfgfgfg_test2
<class 'numpy.ndarray'> [1.6  2.75]
fggfgfgfg_test2


In [6]:
C22.shape

(3432, 31)

In [7]:
Clist   = ['CO2(ppm)', 'CH4(ppb)', 'N2O(ppb)', 'CF4(ppt)', 'C2F6(ppt)', 'C6F14(ppt)', 'HFC23(ppt)', 'HFC32(ppt)', 'HFC43_10(ppt)', 'HFC125(ppt)',
            'HFC134A(ppt)', 'HFC143A(ppt)', 'HFC227EA(ppt)', 'HFC245FA(ppt)', 'SF6(ppt)', 'CFC11(ppt)', 'CFC12(ppt)', 'CFC113(ppt)',
            'CFC114(ppt)', 'CFC115(ppt)', 'CARB_TET(ppt)', 'MCF(ppt)','HCFC22(ppt)', 'HCFC141B(ppt)', 'HCFC142B(ppt)',
            'HALON1211(ppt)', 'HALON1202(ppt)', 'HALON1301(ppt)', 'HALON2402(ppt)', 'CH3BR(ppt)', 'CH3CL(ppt)']
Clistc   = ['year','CO2(ppm)', 'CH4(ppb)', 'N2O(ppb)', 'CF4(ppt)', 'C2F6(ppt)', 'C6F14(ppt)', 'HFC23(ppt)', 'HFC32(ppt)', 'HFC43_10(ppt)', 'HFC125(ppt)',
            'HFC134A(ppt)', 'HFC143A(ppt)', 'HFC227EA(ppt)', 'HFC245FA(ppt)', 'SF6(ppt)', 'CFC11(ppt)', 'CFC12(ppt)', 'CFC113(ppt)',
            'CFC114(ppt)', 'CFC115(ppt)', 'CARB_TET(ppt)', 'MCF(ppt)','HCFC22(ppt)', 'HCFC141B(ppt)', 'HCFC142B(ppt)',
            'HALON1211(ppt)', 'HALON1202(ppt)', 'HALON1301(ppt)', 'HALON2402(ppt)', 'CH3BR(ppt)', 'CH3CL(ppt)']


In [8]:
Elist=['year','co2_fossil(GtC/year)','co2_land(GtC/year)','ch4(Mt/year)','n2o(Mt(N2)/year)','sox(MtS/year)','co(Mt/year)','nmvoc(Mt/year)','nox(MtN/year)','bc(Mt/year)',
       'oc(Mt/year)','nh3(Mt/year)','cf4(kt/year)',
       'c2f6(kt/year)','c6f14(kt/year)','hfc23(kt/year)','hfc32(kt/year)','hfc43_10(kt/year)',
       'hfc125(kt/year)','hfc134a(kt/year)','hfc143a(kt/year)','hfc227ea(kt/year)',
       'hfc245fa(kt/year)','sf6(kt/year)','cfc11(kt/year)','cfc12(kt/year)','cfc113(kt/year)',
       'cfc114(kt/year)','cfc115(kt/year)','carb_tet(kt/year)','mcf(kt/year)','hcfc22(kt/year)',
       'hcfc141b(kt/year)','hcfc142b(kt/year)','halon1211(kt/year)','halon1202(kt/year)',
       'halon1301(kt/year)','halon2402(kt/year)','ch3br(kt/year)','ch3cl(kt/year)']

In [9]:
Flist=['CO2(Wm-2)','CH4(Wm-2)','N2O(Wm-2)','OtherGHG(Wm-2)','Trop_O3(Wm-2)','Strat_O3(Wm-2)','Strat_H2O(Wm-2)','Contrails(Wm-2)',
       'total_aerosol(Wm-2)','bc_snow(Wm-2)','land_use_alb(Wm-2)','volcanic(Wm-2)','solar(Wm-2)']
Flistc=['year','CO2(Wm-2)','CH4(Wm-2)','N2O(Wm-2)','OtherGHG(Wm-2)','Trop_O3(Wm-2)','Strat_O3(Wm-2)','Strat_H2O(Wm-2)','Contrails(Wm-2)',
       'total_aerosol(Wm-2)','bc_snow(Wm-2)','land_use_alb(Wm-2)','volcanic(Wm-2)','solar(Wm-2)']



In [10]:
len(Elist)

40

In [11]:
emissions_base[:,0]

array([1765.04166667, 1765.125     , 1765.20833333, ..., 2050.79166667,
       2050.875     , 2050.95833333])

In [12]:
dfF=pd.DataFrame(columns=Flist,data=Fbase)
dfC=pd.DataFrame(columns=Clist,data=Cbase)
dfE=pd.DataFrame(columns=Elist,data=emissions_base)
dfF['year']=emissions_base[:,0]
dfC['year']=emissions_base[:,0]
dfF.set_index('year')
dfC.set_index('year')
with pd.ExcelWriter('Base_pathway.xlsx') as writer:
    dfE.to_excel(writer,
             sheet_name='Emissions')
    dfC.to_excel(writer,
             sheet_name='Concentrations',columns=Clistc)
    dfF.to_excel(writer,
             sheet_name='Radiative Forcing',columns=Flistc)

In [13]:
dfF=pd.DataFrame(columns=Flist,data=F3)
dfC=pd.DataFrame(columns=Clist,data=C3)
dfE=pd.DataFrame(columns=Elist,data=emissions_g3)
dfF['year']=emissions_base[:,0]
dfC['year']=emissions_base[:,0]
dfF.set_index('year')
dfC.set_index('year')
with pd.ExcelWriter('TwoYearBlip_pathway.xlsx') as writer:
    dfE.to_excel(writer,
             sheet_name='Emissions')
    dfC.to_excel(writer,
             sheet_name='Concentrations',columns=Clistc)
    dfF.to_excel(writer,
             sheet_name='Radiative Forcing',columns=Flistc)

In [14]:
dfF=pd.DataFrame(columns=Flist,data=F22)
dfC=pd.DataFrame(columns=Clist,data=C22)
dfE=pd.DataFrame(columns=Elist,data=emissions_g2_2)
dfF['year']=emissions_base[:,0]
dfC['year']=emissions_base[:,0]
dfF.set_index('year')
dfC.set_index('year')
with pd.ExcelWriter('FossilFuel_pathway.xlsx') as writer:
    dfE.to_excel(writer,
             sheet_name='Emissions')
    dfC.to_excel(writer,
             sheet_name='Concentrations',columns=Clistc)
    dfF.to_excel(writer,
             sheet_name='Radiative Forcing',columns=Flistc)

In [15]:
dfF=pd.DataFrame(columns=Flist,data=F13)
dfC=pd.DataFrame(columns=Clist,data=C13)
dfE=pd.DataFrame(columns=Elist,data=emissions_g1_3)
dfF['year']=emissions_base[:,0]
dfC['year']=emissions_base[:,0]
dfF.set_index('year')
dfC.set_index('year')
with pd.ExcelWriter('ModerateGreen_pathway.xlsx') as writer:
    dfE.to_excel(writer,
             sheet_name='Emissions')
    dfC.to_excel(writer,
             sheet_name='Concentrations',columns=Clistc)
    dfF.to_excel(writer,
             sheet_name='Radiative Forcing',columns=Flistc)

In [16]:
dfF=pd.DataFrame(columns=Flist,data=F4)
dfC=pd.DataFrame(columns=Clist,data=C4)
dfE=pd.DataFrame(columns=Elist,data=emissions_g4)
dfF['year']=emissions_base[:,0]
dfC['year']=emissions_base[:,0]
dfF.set_index('year')
dfC.set_index('year')
with pd.ExcelWriter('StrongGreen_pathway.xlsx') as writer:
    dfE.to_excel(writer,
             sheet_name='Emissions')
    dfC.to_excel(writer,
             sheet_name='Concentrations',columns=Clistc)
    dfF.to_excel(writer,
             sheet_name='Radiative Forcing',columns=Flistc)